# Данные аналитики


https://docs.ozon.ru/api/seller/#operation/AnalyticsAPI_AnalyticsGetData 
Уĸажите период и метриĸи, ĸоторые нужно посчитать. В ответе будет аналитиĸа, сгруппированная по параметру dimensions.

Для продавцов без Premium-подписки:

доступны данные за последние 3 месяца,
есть ограничения по способам группировки данных и метрикам.
Для продавцов с Premium-подпиской ограничений нет.

Метод можно использовать не больше 1 раза в минуту. Соответствует разделу Аналитика → Графики в личном кабинете.

In [10]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import date, timedelta, datetime
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [11]:
# Retrieve API keys from environment variables
KeyGuten_Ozon = os.getenv('KeyGuten_Ozon')
IdGuten_Ozon = os.getenv('IdGuten_Ozon')


In [ ]:
class OzonAnalyticsClient:
    def __init__(self, client_id, api_key):
        self.base_url = "https://api-seller.ozon.ru"
        self.headers = {
            "Client-Id": IdGuten_Ozon,
            "Api-Key":KeyGuten_Ozon,
            "Content-Type": "application/json"
        }

    def get_analytics_data(self, date_from, date_to, dimensions, metrics, limit, 
                         sort):
        """
        Get analytics data from Ozon API
        
        Args:
            date_from (str): Start date in YYYY-MM-DD format
            date_to (str): End date in YYYY-MM-DD format
            dimensions (list): List of dimension strings (e.g., ['day', 'sku'])
            metrics (list): List of metric strings (e.g., ['revenue', 'ordered_units'])
            limit (int): Number of results (1-1000)
            sort (list, optional): List of sort objects
            
        Returns:
            dict: API response
        """
        endpoint = "/v1/analytics/data"
        url = self.base_url + endpoint
        
        # Validate dates
        try:
            datetime.strptime(date_from, "%Y-%m-%d")
            datetime.strptime(date_to, "%Y-%m-%d")
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format")
        
        # Prepare request body
        payload = {
            "date_from": date_from,
            "date_to": date_to,
            "dimension": dimensions,
            "metrics": metrics,
            "limit": limit
        }

        if sort is not None:
            payload["sort"] = sort
            
        try:
            response = requests.post(url, headers=self.headers, data=json.dumps(payload))
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error making request: {e}")
            return None

# Example usage
if __name__ == "__main__":
    # Initialize client with your credentials
    CLIENT_ID = IdGuten_Ozon
    API_KEY = KeyGuten_Ozon
    client = OzonAnalyticsClient(CLIENT_ID, API_KEY)
    
    # Prepare request parameters
    request_params = {
        "date_from": "2025-01-01",
        "date_to": "2025-03-23",
        "dimensions": ["day", "sku"],
        "metrics": ["revenue", "ordered_units","hits_view_search", "hits_view_pdp", "hits_view", "hits_tocart_search", "hits_tocart_pdp", 
                    "hits_tocart", "session_view_search", "session_view_pdp", "session_view", "conv_tocart_search", "conv_tocart_pdp", "conv_tocart",
                    "returns", "cancellations", "delivered_units", "position_category"],
        "limit" : 1000,
        "sort": [{"key": "revenue", "order": "DESC"}]
    }
    
    # Get analytics data
    analytics_data = client.get_analytics_data(**request_params)
    
    if analytics_data:
        print("Successfully retrieved analytics data:")
        print(json.dumps(analytics_data, indent=2, ensure_ascii=False))
    else:
        print("Failed to retrieve analytics data")

Error making request: 400 Client Error: Bad Request for url: https://api-seller.ozon.ru/v1/analytics/data
Failed to retrieve analytics data


"hits_view_search", "hits_view_pdp", "hits_view", "hits_tocart_search", "hits_tocart_pdp", "hits_tocart", "session_view_search", "session_view_pdp", "session_view", "conv_tocart_search", "conv_tocart_pdp", "conv_tocart", "returns", "cancellations", "delivered_units", "position_category"